In [1]:
import numpy as np
import pandas
import re
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

#from pymystem3 import Mystem
import pymorphy2

# инициализируем
vectorizer = CountVectorizer()
#m = Mystem()
morph = pymorphy2.MorphAnalyzer()

In [2]:
# функция для лемматизации

def lemmatize(text):
    text_str = str(text)
    text_str = re.sub(r'[^\w\s]+|[\d]+', r'',text_str).strip()
    words = text_str.split() # разбиваем текст на слова
    res = ""
    for word in words:
        p = morph.parse(word)[0]
        res = res + " " + p.normal_form

    return res

In [3]:
data_xls = pandas.read_excel("resolution_100.xlsx")
data_xls

text  priznak
0     Данный документ был направлен с целью исполнения.        0
1     Прошу  проверить, при отсутствии замечаний при...        0
2                   прошу заключить договор НМТП + НСРЗ        1
3     проработать, подготовить ответ. \nпроработать,...        1
4     Рассмотреть обращение. дать позицию ЕКД. прове...        1
...                                                 ...      ...
1491                           см. файл с комментариями        0
1492  проработать\n1. в качестве замещении выпадающи...        0
1493                                  что можете взять?        0
1494  Станислав, проработай заявку с ПД и подготовь ...        1
1495  прошу провести проверку фактов, изложенных в С...        0

[1496 rows x 2 columns]

In [4]:
# проводим лемматизацию
data_xls['text'] = data_xls.apply(lambda row: lemmatize(row['text']), axis=1)

In [22]:
pandas.options.display.width=500

In [5]:
data_xls

text  priznak
0      данный документ быть направить с цель исполнение        0
1      просить проверить при отсутствие замечание пр...        0
2               просить заключить договор нмтп нсрзнуть        1
3      проработать подготовить ответ проработать под...        1
4      рассмотреть обращение дать позиция екд провер...        1
...                                                 ...      ...
1491                       сантиметр файл с комментарий        0
1492   проработать в качество замещение выпадать объ...        0
1493                                     что мочь взять        0
1494   станислав проработать заявка с пд и подготови...        1
1495   просить провести проверка факт изложить в сз ...        0

[1496 rows x 2 columns]

In [5]:
text = data_xls.text

In [30]:
type(text)
text

0        данный документ быть направить с цель исполнение
1        просить проверить при отсутствие замечание пр...
2                 просить заключить договор нмтп нсрзнуть
3        проработать подготовить ответ проработать под...
4        рассмотреть обращение дать позиция екд провер...
                              ...                        
1491                         сантиметр файл с комментарий
1492     проработать в качество замещение выпадать объ...
1493                                       что мочь взять
1494     станислав проработать заявка с пд и подготови...
1495     просить провести проверка факт изложить в сз ...
Name: text, Length: 1496, dtype: object

In [6]:
# создаем векторы
text_vector = vectorizer.fit_transform(text)

In [7]:
# чтобы получить сгенерированный словарь, из приведенной структуры CountVectorizer, стоит отметить что порядок совпадает с матрицей
vectorizer.get_feature_names()  # ['слово1', 'слово2', 'слово3', 'слово4']

['east',
 'hhru',
 'metals',
 'nan',
 'pзадача',
 'wrike',
 'аа',
 'ава',
 'аванс',
 'авр',
 'автомашина',
 'автотранспорт',
 'авцыганка',
 'агент',
 'адрес',
 'аи',
 'аккуратный',
 'акт',
 'актуализировать',
 'актуальность',
 'акцепт',
 'александр',
 'алексей',
 'алиев',
 'алкогольный',
 'алл',
 'алюминий',
 'ан',
 'анализ',
 'аналогичный',
 'аналогия',
 'анкета',
 'аннулирование',
 'аннулировать',
 'ао',
 'апрель',
 'аренда',
 'арендатор',
 'артамонов',
 'архив',
 'ас',
 'ая',
 'база',
 'базовый',
 'банк',
 'банкагарант',
 'барсук',
 'барсуков',
 'бг',
 'без',
 'белянский',
 'бесплатный',
 'битум',
 'ближний',
 'более',
 'большой',
 'боровок',
 'боровоко',
 'бородовицын',
 'бп',
 'бск',
 'бу',
 'бункеровка',
 'буссель',
 'бухгалтер',
 'бухгалтерия',
 'быстрый',
 'быть',
 'бюджет',
 'бюджетный',
 'вагон',
 'вакансия',
 'вакуумный',
 'валерий',
 'вариант',
 'ваш',
 'вводный',
 'ввоз',
 'ведение',
 'вера',
 'вернуть',
 'вернуться',
 'весь',
 'взаимодействие',
 'взаиморасчёт',
 'взять',


In [8]:
# показать матрицу
text_vector_array = text_vector.toarray()
print(text_vector_array)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [9]:
len(text_vector_array[0])

1336

In [11]:
past_columns = [] # пустой массив, для опрееделени колонок
for i in range(len(text_vector_array[0])): 
    past_columns.append(f"v_{i}")
past_columns

['v_0',
 'v_1',
 'v_2',
 'v_3',
 'v_4',
 'v_5',
 'v_6',
 'v_7',
 'v_8',
 'v_9',
 'v_10',
 'v_11',
 'v_12',
 'v_13',
 'v_14',
 'v_15',
 'v_16',
 'v_17',
 'v_18',
 'v_19',
 'v_20',
 'v_21',
 'v_22',
 'v_23',
 'v_24',
 'v_25',
 'v_26',
 'v_27',
 'v_28',
 'v_29',
 'v_30',
 'v_31',
 'v_32',
 'v_33',
 'v_34',
 'v_35',
 'v_36',
 'v_37',
 'v_38',
 'v_39',
 'v_40',
 'v_41',
 'v_42',
 'v_43',
 'v_44',
 'v_45',
 'v_46',
 'v_47',
 'v_48',
 'v_49',
 'v_50',
 'v_51',
 'v_52',
 'v_53',
 'v_54',
 'v_55',
 'v_56',
 'v_57',
 'v_58',
 'v_59',
 'v_60',
 'v_61',
 'v_62',
 'v_63',
 'v_64',
 'v_65',
 'v_66',
 'v_67',
 'v_68',
 'v_69',
 'v_70',
 'v_71',
 'v_72',
 'v_73',
 'v_74',
 'v_75',
 'v_76',
 'v_77',
 'v_78',
 'v_79',
 'v_80',
 'v_81',
 'v_82',
 'v_83',
 'v_84',
 'v_85',
 'v_86',
 'v_87',
 'v_88',
 'v_89',
 'v_90',
 'v_91',
 'v_92',
 'v_93',
 'v_94',
 'v_95',
 'v_96',
 'v_97',
 'v_98',
 'v_99',
 'v_100',
 'v_101',
 'v_102',
 'v_103',
 'v_104',
 'v_105',
 'v_106',
 'v_107',
 'v_108',
 'v_109',
 'v_110',


In [12]:
dataframe_vector = pandas.DataFrame(data = text_vector_array,columns = past_columns)
dataframe_vector

v_0  v_1  v_2  v_3  v_4  v_5  v_6  v_7  v_8  v_9  ...  v_1326  v_1327  \
0       0    0    0    0    0    0    0    0    0    0  ...       0       0   
1       0    0    0    0    0    0    0    0    0    0  ...       0       0   
2       0    0    0    0    0    0    0    0    0    0  ...       0       0   
3       0    0    0    0    0    0    0    0    0    0  ...       0       0   
4       0    0    0    0    0    0    0    0    0    0  ...       0       0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...     ...     ...   
1491    0    0    0    0    0    0    0    0    0    0  ...       0       0   
1492    0    0    0    0    0    0    0    0    0    0  ...       0       0   
1493    0    0    0    0    0    0    0    0    0    0  ...       0       0   
1494    0    0    0    0    0    0    0    0    0    0  ...       0       0   
1495    0    0    0    0    0    0    0    0    0    0  ...       0       0   

      v_1328  v_1329  v_1330  v_1331  v_1332  v_1333  v_1334  v_1335  
0          0       0       0       0       0       0       0       0  
1          0       0       0       0       0       0       0       0  
2          0       0       0       0       0       0       0       0  
3          0       0       0       0       0       0       0       0  
4          0       0       0       0       0       0       0       0  
...      ...     ...     ...     ...     ...     ...     ...     ...  
1491       0       0       0       0       0       0       0       0  
1492       0       0       0       0       0       0       0       0  
1493       0       0       0       0       0       0       0       0  
1494       0       0       0       0       0       0       0       0  
1495       0       0       0       0       0       0       0       0  

[1496 rows x 1336 columns]

In [13]:
data = pandas.concat([data_xls,dataframe_vector],axis = 1)


In [43]:
data[data["v_1328"]>0]

text  priznak  v_0  v_1  \
945    рассмотреть необходимость участие юрист правупра        0    0    0   
1415   катерина просить приложить сз от юрист с реко...        1    0    0   

      v_2  v_3  v_4  v_5  v_6  v_7  ...  v_1326  v_1327  v_1328  v_1329  \
945     0    0    0    0    0    0  ...       0       0       1       0   
1415    0    0    0    0    0    0  ...       0       0       1       0   

      v_1330  v_1331  v_1332  v_1333  v_1334  v_1335  
945        0       0       0       0       0       0  
1415       0       0       0       0       0       0  

[2 rows x 1338 columns]

In [14]:
Y = data.priznak
X = data[past_columns]

In [15]:
# предсказание простое
clf = RandomForestClassifier(random_state=241)
clf.fit(X, Y)
# расчет веса показателей
importances = clf.feature_importances_
print(X.columns)
print(importances)

Index(['v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9',
       ...
       'v_1326', 'v_1327', 'v_1328', 'v_1329', 'v_1330', 'v_1331', 'v_1332',
       'v_1333', 'v_1334', 'v_1335'],
      dtype='object', length=1336)
[2.73366444e-04 1.87508486e-05 4.34693129e-04 ... 2.30440080e-04
 9.40373635e-05 1.57198623e-05]


In [94]:
clf = RandomForestClassifier()

# здесь пишем какие параметры будет перебирать

param_greed = {
    "max_depth" : [2,5,10],
    "criterion" : ['gini', 'entropy'],
    "min_samples_split" : [2,5,10],
    "min_samples_leaf" : [1,5,10],
}

In [95]:
# пишем каие параметры перебираем scoring = как оцениваем  ,
# cv = сколько тестовых выборок, n_jobs = -1 используем все процссоры
#GS = GridSearchCV(clf,param_greed,scoring = 'roc_auc',cv = 6,n_jobs = -1)
GS = GridSearchCV(clf,param_greed,scoring = 'roc_auc',cv = 6,n_jobs = -1)

In [96]:
GS.fit(X,Y)

GridSearchCV(cv=6, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [97]:
# лучший показатель
GS.best_score_

0.9736540311933585

In [98]:
GS.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'min_samples_leaf': 1,
 'min_samples_split': 5}

In [18]:
# ДЛЯ ПРЯМОГО РАСЧЕТА
# теперь можно быстро подсчитать вектор для нового документа
#text2 = ["the puppy"]
#vector = vectorizer.transform(text2)
#print(vector.toarray())

text_predict = 'Подготовить отчет'
text_predict_lemmat = lemmatize(text_predict)
new_v = vectorizer.transform([text_predict_lemmat])  # результат [[1 0 0 2]]
new_v_array = new_v.toarray()
dataframe_predict = pandas.DataFrame(data = new_v_array,columns = past_columns)
print(dataframe_predict)
clf.predict(dataframe_predict)

   v_0  v_1  v_2  v_3  v_4  v_5  v_6  v_7  v_8  v_9  ...  v_1326  v_1327  \
0    0    0    0    0    0    0    0    0    0    0  ...       0       0   

   v_1328  v_1329  v_1330  v_1331  v_1332  v_1333  v_1334  v_1335  
0       0       0       0       0       0       0       0       0  

[1 rows x 1336 columns]


array([1])

In [117]:
# СОХРАНЕНИЕ МОДЕЛИ
filename = 'finalized_model_resolut.sav'
pickle.dump(clf, open(filename, 'wb'))

In [14]:
# ЗАГРУЗКА МОДЕЛИ
filename = 'finalized_model_resolut.sav'
loaded_clf = pickle.load(open(filename, 'rb'))

In [25]:
# СОХРАНЕНИЕ ВЕКТОРОВ
filename = 'finalized_vector_resolut.sav'
pickle.dump(vectorizer, open(filename, 'wb'))

In [26]:
# ЗАГРУЗКА ВЕКТОРОВ
filename = 'finalized_vector_resolut.sav'
loaded_vectorizer = pickle.load(open(filename, 'rb'))

In [27]:
# РАСЧЕТ С ЗАГРУЖЕННОЙ МОДЕЛИ
# теперь можно быстро подсчитать вектор для нового документа
#text2 = ["the puppy"]
#vector = vectorizer.transform(text2)
#print(vector.toarray())

text_predict = 'предоставьте ответ 44'
text_predict_lemmat = lemmatize(text_predict)
new_v = vectorizer.transform([text_predict_lemmat])  # результат [[1 0 0 2]]
new_v_array = new_v.toarray()
dataframe_predict = pandas.DataFrame(data = new_v_array,columns = past_columns)
print(dataframe_predict)
loaded_clf.predict(dataframe_predict)

   v_0  v_1  v_2  v_3  v_4  v_5  v_6  v_7  v_8  v_9  ...  v_1326  v_1327  \
0    0    0    0    0    0    0    0    0    0    0  ...       0       0   

   v_1328  v_1329  v_1330  v_1331  v_1332  v_1333  v_1334  v_1335  
0       0       0       0       0       0       0       0       0  

[1 rows x 1336 columns]


array([1])

In [101]:
# ДЛЯ РАСЧЕТА ЧЕРЕЗ GS
# теперь можно быстро подсчитать вектор для нового документа
#text2 = ["the puppy"]
#vector = vectorizer.transform(text2)
#print(vector.toarray())

text_predict = 'в приказ'
text_predict_lemmat = lemmatize(text_predict)
new_v = vectorizer.transform([text_predict_lemmat])  # результат [[1 0 0 2]]
new_v_array = new_v.toarray()
dataframe_predict = pandas.DataFrame(data = new_v_array,columns = past_columns)
print(dataframe_predict)
GS.best_estimator_.predict(dataframe_predict)

   v_0  v_1  v_2  v_3  v_4  v_5  v_6  v_7  v_8  v_9  ...  v_1326  v_1327  v_1328  v_1329  v_1330  v_1331  v_1332  v_1333  v_1334  v_1335
0    0    0    0    0    0    0    0    0    0    0  ...       0       0       0       0       0       0       0       0       0       0

[1 rows x 1336 columns]


array([0])